In [1]:
import os, shutil
import glob
import json
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imagesize


### 根据CSV文件的数据进行标注

In [ ]:
"""
Format Train and Test Set
"""
BASE_DATA_PATH = "/home/tipriest/data/seg_risky"
YOLO_SEG_LABEL_BASE_PATH = os.path.join(BASE_DATA_PATH, "datasetLabel")
YOLO_SEG_LABEL_TRAIN_PATH = os.path.join(YOLO_SEG_LABEL_BASE_PATH, "train/")
YOLO_SEG_LABEL_VALID_PATH = os.path.join(YOLO_SEG_LABEL_BASE_PATH, "valid/")
YOLO_SEG_LABEL_TEST_PATH = os.path.join(YOLO_SEG_LABEL_BASE_PATH, "test/")

# training_anno = pd.read_csv(os.path.join(
#     BASE_DATA_PATH, "seg_risky_training_anno.csv"))
test_anno = pd.read_csv("../Yolo/track2_submit_15_30.csv")
# for i in range(1, 2, 1):
#     train_jpgs += [x.replace('/home/tipriest/data/seg_risky/', '')
#                    for x in glob.glob(os.path.join(BASE_DATA_PATH, f'{i}/*.jpg'))]
test_jpgs = glob.glob(os.path.join(BASE_DATA_PATH, "test_set_A_rename/*/*"))
test_jpgs2 = []
for test_jpg in test_jpgs:
    test_jpgs2.append(test_jpg.split('/')[-1])
test_anno = test_anno[test_anno['Path'].isin(test_jpgs2)]
test_anno['Polygon'] = test_anno['Polygon'].apply(json.loads)


if os.path.exists(YOLO_SEG_LABEL_BASE_PATH):
    shutil.rmtree(YOLO_SEG_LABEL_BASE_PATH)
os.makedirs(YOLO_SEG_LABEL_TRAIN_PATH)
os.makedirs(YOLO_SEG_LABEL_VALID_PATH)
os.makedirs(YOLO_SEG_LABEL_TEST_PATH)

for index, row in test_anno.iloc[:100].iterrows():
    # shutil.copy(os.path.join(BASE_DATA_PATH,
    #             row[1].Path), YOLO_SEG_LABEL_TRAIN_PATH)
    img = cv2.imread(test_jpgs[index])
    if img is None:
        print("Error: 无法读取图像 ", row[1].Path)
    img_height, img_width = img.shape[:2]
    # 在图像上绘制标注
    row1 = row[1]
    for polygon in row[1]:
        pts = np.array(polygon, np.int32)
        pts = pts.reshape((-1, 1, 2))
        img = cv2.polylines(img, [pts], isClosed=True, color=(
            0, 255, 0), thickness=3)  # 绘制绿色多边形
        file_name = test_jpgs[index].split('/')[-1]  # 获取文件名部分
        file_name_without_prefix = file_name.split(
            ".")[-2][-5:]  # 获取文件名部分（不包含后缀）
        # cv2.waitKey(500)
        result = cv2.imwrite(os.path.join(YOLO_SEG_LABEL_TEST_PATH,
                                          f'{file_name_without_prefix}.jpg'), img)
        # result = cv2.imwrite("~/data/test.jpg", img)
        if not result:
            print(f'{file_name_without_prefix}.jpg, 保存失败')


### 根据生成的YOLO标签进行标注

In [ ]:
# def yolo2coco(W, H, pts):
#     for label in labelList:
#     label = label.strip().split()
#     x = float(label[1])
#     y = float(label[2])
#     w = float(label[3])
#     h = float(label[4])

#     # convert x,y,w,h to x1,y1,x2,y2
#     H, W, _ = im.shape
#     x1 = (x - w / 2) * W
#     y1 = (y - h / 2) * H
#     x2 = (x + w / 2) * W
#     y2 = (y + h / 2) * H
"""
Format Train and Test Set
"""
BASE_DATA_PATH = "/home/tipriest/data/seg_risky/datasetCOCO"
COCO_TRAIN_LABEL_PATH = os.path.join(BASE_DATA_PATH, "train/labels")
COCO_TRAIN_IMAGE_PATH = os.path.join(BASE_DATA_PATH, "train/images")
COCO_TRAIN_LABELLED_PATH = os.path.join(BASE_DATA_PATH, "train/labelled")
os.makedirs(COCO_TRAIN_LABELLED_PATH, exist_ok=True)

LABEL_FILE_LIST = [f for f in os.listdir(COCO_TRAIN_LABEL_PATH) if f.endswith(".txt")]
for f in LABEL_FILE_LIST:
    file_path = os.path.join(COCO_TRAIN_LABEL_PATH, f)
    image_path = os.path.join(COCO_TRAIN_IMAGE_PATH, f.replace(".txt", ".jpg"))
    [w, h] = imagesize.get(image_path)

    with open(file_path, 'r') as file:
        contents = file.read().split()
    pts = []
    for i in range(len(contents)):
        # print(f"i = {i}, contents[i] = {contents[i]}")
        if 0 == i % 9:
            continue
        if 1 == i % 2:
            pts.append(float(contents[i])*w)
        else:
            pts.append(float(contents[i])*h)
    pts = np.array(pts, np.int32)
    pts = pts.reshape((-1, 1, 2))
    # print(pts)
    img = cv2.imread(filename=image_path)
    img = cv2.polylines(img, [pts], isClosed=True, color=(
        0, 255, 0), thickness=3)  # 绘制绿色多边形
    # cv2.imshow("img", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    f_noprefix = f.split(".")[-2][-5:]  # 获取文件名部分(不包含后缀)
    res = cv2.imwrite(os.path.join(COCO_TRAIN_LABELLED_PATH, f"{f_noprefix}.jpg"), img)
    # print(w, h)
    # img = cv2.imread(image_path)
# with open(txt_filename, 'w') as up:
#     for polygon in row[1].Polygons:
#         normalized_polygon = normalize_polygon(
#             polygon, img_width, img_height)
#         normalized_coords = ' '.join(
#             [f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
#         up.write(f'0 {normalized_coords}\n')
# # training_anno = pd.read_csv(os.path.join(
# #     BASE_DATA_PATH, "seg_risky_training_anno.csv"))
# test_anno = pd.read_csv("../Yolo/track2_submit_15_30.csv")
# # for i in range(1, 2, 1):
# #     train_jpgs += [x.replace('/home/tipriest/data/seg_risky/', '')
# #                    for x in glob.glob(os.path.join(BASE_DATA_PATH, f'{i}/*.jpg'))]
# test_jpgs = glob.glob(os.path.join(BASE_DATA_PATH, "test_set_A_rename/*/*"))
# test_jpgs2 = []
# for test_jpg in test_jpgs:
#     test_jpgs2.append(test_jpg.split('/')[-1])
# test_anno = test_anno[test_anno['Path'].isin(test_jpgs2)]
# test_anno['Polygon'] = test_anno['Polygon'].apply(json.loads)


# if os.path.exists(YOLO_SEG_LABEL_BASE_PATH):
#     shutil.rmtree(YOLO_SEG_LABEL_BASE_PATH)
# os.makedirs(YOLO_SEG_LABEL_TRAIN_PATH)
# os.makedirs(YOLO_SEG_LABEL_VALID_PATH)
# os.makedirs(YOLO_SEG_LABEL_TEST_PATH)

# for index, row in test_anno.iloc[:100].iterrows():
#     # shutil.copy(os.path.join(BASE_DATA_PATH,
#     #             row[1].Path), YOLO_SEG_LABEL_TRAIN_PATH)
#     img = cv2.imread(test_jpgs[index])
#     if img is None:
#         print("Error: 无法读取图像 ", row[1].Path)
#     img_height, img_width = img.shape[:2]
#     # 在图像上绘制标注
#     row1 = row[1]
#     for polygon in row[1]:
#         pts = np.array(polygon, np.int32)
#         pts = pts.reshape((-1, 1, 2))
#         img = cv2.polylines(img, [pts], isClosed=True, color=(
#             0, 255, 0), thickness=3)  # 绘制绿色多边形
#         file_name = test_jpgs[index].split('/')[-1]  # 获取文件名部分
#         file_name_without_prefix = file_name.split(
#             ".")[-2][-5:]  # 获取文件名部分(不包含后缀)
#         # cv2.waitKey(500)
#         result = cv2.imwrite(os.path.join(YOLO_SEG_LABEL_TEST_PATH,
#                                             f'{file_name_without_prefix}.jpg'), img)
#         # result = cv2.imwrite("~/data/test.jpg", img)
#         if not result:
#             print(f'{file_name_without_prefix}.jpg, 保存失败')


### 根据生成的COCO标签进行标注

In [8]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm
# 读取COCO格式的标注文件
def load_annotations(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        annotations = json.load(f)
    return annotations

# 创建保存图片的文件夹
def create_output_folder(output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

# 画出多边形并保存图片
def draw_polygons_and_save(annotations, img_dir, output_dir):
    create_output_folder(output_dir)
    
    # 遍历所有的标注
    for ann in tqdm(annotations['annotations']):
        # 获取图片ID和文件名
        image_id = ann['image_id']
        image_info = next(img for img in annotations['images'] if img['id'] == image_id)
        file_name = image_info['file_name']
        img_path = os.path.join(img_dir, file_name)
        
        # 读取图片
        img = cv2.imread(img_path)
        if img is None:
            print(f"无法读取图片: {img_path}")
            continue
        # 获取segmentation (多边形坐标)
        if 'segmentation' in ann:
            for segmentation in ann['segmentation']:
                # 将segmentation变成一个二维数组
                pts = np.array(segmentation).reshape((-1, 1, 2)).astype(np.int32)
                # 在图片上绘制多边形
                cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

        # 保存标注后的图片
        output_path = os.path.join(output_dir, file_name)
        res = cv2.imwrite(output_path, img)
        if not res:
            print("False")

# 主函数
# 文件路径设置
json_path = "/home/tipriest/data/seg_risky/datasetCOCO/train/annotations/train_anno.json"  # COCO的标注文件
img_dir = "/home/tipriest/data/seg_risky/datasetCOCO/train/images"  # 图片文件夹
output_dir = "/home/tipriest/data/seg_risky/datasetCOCO/train/labelledcoco"  # 输出文件夹
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
# 读取标注
annotations = load_annotations(json_path)

# 处理图片并保存
draw_polygons_and_save(annotations, img_dir, output_dir)


 17%|█▋        | 13905/82150 [00:46<03:47, 299.48it/s]


KeyboardInterrupt: 